In [1]:
from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np

ctrl = Controller(*init_config())

2021-12-27 19:07:34.253357: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir
Namespace(act='elu', action_gnn_hidden_act=64, action_gnn_hidden_val=200, action_gnn_layers=6, action_gnn_next_shape=(32, 64, 32), action_gnn_start_shape=(4, 1, 4), action_repeat=1, actor_disc=5, actor_dist='normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=0.001, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, atari_grayscale=False, behavior_stop_grad=True, clip_rewards='identity', collect_dyn_sample=True, dataset_size=100000, debug=False, disag_layers=1, disag_log=True, disag_models=3, disag_offset=0, disag_target='embed', disag_units=200, discount=0.95, discount_lambda=0.8, discount_layers=2, discount_scale=1.0, dyn_cell='gru', dyn_deter=512, dyn_discrete=32, dyn_hidden=512, dyn_input_layers=5, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_lay

usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir


Logdir logdir


2021-12-27 19:07:35.639454: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-27 19:07:35.640077: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-27 19:07:36.913284: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 19:07:36.913920: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 19:07:36.913940: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-27 19:07:36.915274: I

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t111_funct_2.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t38_waybel_9.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Prefill dataset (0 steps).
Loaded problem: m2n140__t95_funct_2
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t2_wellord1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Eval episode has 25 steps and return -0.500.


/home/kristsz/DreamProLog/envs/wrappers.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  value = np.array(value)


In [2]:
meta = ctrl.datasetManager._train_eps._meta
ds = []
for v in meta.values():
    ds.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    return np.pad(narr, [0, 128-size])

parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [3]:
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    for i in range(1000):
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x, x
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

2021-12-27 19:07:47.879130: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-27 19:07:48.103498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 19:07:48.104193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 19:07:48.104217: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] 

In [4]:
from unorganized.parts3 import Net

In [5]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.0):
        super().__init__()
        self.omega = omega
        
    def call(self, y_true, y_pred):
        y = tf.nn.softmax(y_pred, axis=-1)
        y_probs = tf.gather(y, y_true, axis=-1, batch_dims=2)
        y_log_probs = tf.math.log(y_probs+0.001)
        #loss = self.omega*y_log_probs + (1-self.omega)*tf.math.cumsum(y_log_probs, axis=-1)
        #loss = y_log_probs*tf.math.cumprod(0.97*tf.ones((32,128)), axis=-1)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        #mask = tf.math.cumprod(mask, axis=-1)
        loss = (mask+self.omega)*y_log_probs
        loss = -tf.math.reduce_sum(loss)/(128*32)
        return loss

In [6]:
model = Net(3, 8, 128, 16, 1, 32)#Net(4, 8, 128, 64, 4, 128)#Net(128, 4, 256)
#d_model = 256
#enc_embed = tf.keras.layers.Embedding(300, d_model)
#encoder = Encoder(4, d_model, 4, 512, 0.04)

In [7]:
x, x = next(iter(tf_ds))
model(x, False)
y_true = x
mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
mask = tf.math.cumprod(mask, axis=-1)

2021-12-27 19:07:48.865484: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-27 19:07:48.885478: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3598190000 Hz
2021-12-27 19:07:48.911607: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-27 19:07:49.069523: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=Loss(), metrics=[])


In [9]:
model.fit(tf_ds, epochs = 50)

Epoch 1/50
1000/1000 [==============================] - 9s 8ms/step - loss: 0.4382
Epoch 2/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0142
Epoch 3/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0043
Epoch 4/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0017
Epoch 5/50
1000/1000 [==============================] - 8s 8ms/step - loss: 7.1228e-04
Epoch 6/50
1000/1000 [==============================] - 8s 8ms/step - loss: 2.9062e-04
Epoch 7/50
1000/1000 [==============================] - 8s 8ms/step - loss: 1.0895e-04
Epoch 8/50
1000/1000 [==============================] - 8s 8ms/step - loss: -4.5364e-06
Epoch 9/50
1000/1000 [==============================] - 8s 8ms/step - loss: -6.0888e-05
Epoch 10/50
1000/1000 [==============================] - 8s 8ms/step - loss: -1.1322e-04
Epoch 11/50
1000/1000 [==============================] - 8s 8ms/step - loss: -1.3569e-04
Epoch 12/50
 126/1000 [==>...........................

KeyboardInterrupt: 

In [10]:

x, y_true = next(iter(tf_ds))
y_pred = model(x, False)
y_pred = tf.nn.softmax(y_pred, axis=-1)
y_probs = tf.gather(y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(y_pred, axis=-1)
print(x[:4], y_pred[:4], tf.cast(100*y_probs[:4], dtype=tf.int32), sep='\n\n')

tf.Tensor(
[[  9   5  12  13   2   3   9   5  16  13   2   9   5  16  12   2   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  1   2  33  23  15   7   2   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   

In [ ]:
model.encoder.variable

In [11]:
mask = tf.cast(tf.math.equal(x[:1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        
v, q = model.enc_embed(x)[:1], model.encoder.variable#tf.tile(model.encoder.variable, (32, 1, 1))
v = [v]
q = [q]
attention = []
for l in model.encoder.layers:
    _v, _q, _att = l(v[-1], q[-1], mask, False)
    v.append(_v)
    q.append(_q)
    attention.append(_att)
    

AttributeError: 'Net' object has no attribute 'encoder'

In [ ]:
v

In [ ]:
attention[0]

In [ ]:
q

In [ ]:
tf.ones((32, 4, 128, 128))

In [ ]:
def eigenRank(tensor):
    batch_shape = tf.shape(tensor)[:-2]
    matrix_shape = tf.shape(tensor)[-2:]
    l = len(batch_shape)
    print(tensor.shape)
    if matrix_shape[0]>matrix_shape[1]: tensor = tf.transpose(tensor, tuple(i for i in range(l)) + (l+1, l))
    M = tf.matmul(tensor, tensor, transpose_b = True)
    print(tensor.shape, M.shape)
    trace = tf.linalg.trace(M)
    M_squared = tf.matmul(M, M)
    print(M_squared.shape)
    trace_squared = tf.linalg.trace(M_squared)
    return (trace**2)/trace_squared

In [ ]:
att = attention[0][0]

In [ ]:
eigenRank(tf.random.uniform(
            (4,8, 128, 128), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=420, name=None))

In [12]:
mask = tf.cast(tf.math.equal(x[:1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        
v = model.enc_embed(x)[:]#, model.encoder.variable#tf.tile(model.encoder.variable, (32, 1, 1))
vs = [v]
att = []
for mha, sl in zip(model.mha_layers, model.sl_layers):
    v, a = mha(v, v, v, mask)
    v = sl(v)
    vs.append(v)
    att.append(a)

In [13]:
z = vs[-1][:8, :1, :16]
z

<tf.Tensor: shape=(8, 1, 16), dtype=float32, numpy=
array([[[ 3.7213912 , -0.7112645 , -4.4756722 ,  1.626818  ,
          0.8056078 ,  0.39228967, -3.0185008 ,  0.31752035,
          1.1245883 , -2.2758284 ,  4.467935  , -3.7383752 ,
          2.628587  ,  2.4846663 , -1.272466  , -0.92475677]],

       [[ 3.746252  , -0.7378752 , -4.436506  ,  1.6601193 ,
          0.6722908 ,  0.30658576, -3.0545259 ,  0.5480538 ,
          1.4075805 , -2.3046896 ,  4.3409038 , -3.8680487 ,
          2.7130494 ,  2.1996963 , -1.282106  , -0.7686895 ]],

       [[ 3.7912145 , -0.727296  , -4.433284  ,  1.6391497 ,
          0.66551423,  0.487097  , -2.9748242 ,  0.38291657,
          0.9985702 , -2.3025029 ,  4.335533  , -3.902844  ,
          2.7539275 ,  2.3473566 , -1.1840688 , -0.73992777]],

       [[ 3.6512928 , -1.3532753 , -4.4880314 ,  2.084999  ,
          0.8288431 ,  0.41406146, -2.8436046 ,  0.59227216,
          0.82978785, -2.008383  ,  4.317675  , -3.7560482 ,
          2.3851287 ,  2

In [14]:
z = z - z[:, :1, :]

In [15]:
tf.reduce_sum(tf.cast(z>0.001, tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [16]:
att[0][:1, :1, :16, :16]

<tf.Tensor: shape=(1, 1, 16, 16), dtype=float32, numpy=
array([[[[0.00639094, 0.00622164, 0.00810817, 0.00718671, 0.0070347 ,
          0.00790245, 0.00639094, 0.00622164, 0.00558972, 0.00718671,
          0.0070347 , 0.00639094, 0.00622164, 0.00558972, 0.00810817,
          0.0070347 ],
         [0.0064485 , 0.00622864, 0.00802322, 0.00725437, 0.00698567,
          0.00793549, 0.0064485 , 0.00622864, 0.00554249, 0.00725437,
          0.00698567, 0.0064485 , 0.00622864, 0.00554249, 0.00802322,
          0.00698567],
         [0.00629238, 0.00614494, 0.00805171, 0.0071684 , 0.00695611,
          0.00796347, 0.00629238, 0.00614494, 0.00541826, 0.0071684 ,
          0.00695611, 0.00629238, 0.00614494, 0.00541826, 0.00805171,
          0.00695611],
         [0.00642512, 0.00625386, 0.00805964, 0.00725787, 0.00703859,
          0.00794166, 0.00642512, 0.00625386, 0.00554446, 0.00725787,
          0.00703859, 0.00642512, 0.00625386, 0.00554446, 0.00805964,
          0.00703859],
         [0.

In [24]:
import tensorflow_probability as tfp

normal = tfp.distributions.Normal(tf.zeros((32)),  tf.ones((32)))
x = tf.cast(tf.math.softmax(normal.sample(24)*tf.constant([[(i+1)*0.4] for i in range(24)]), axis=-1)>0.1, tf.int32)


<tf.Tensor: shape=(24, 32), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0